# Lab 1: Sockets
    
In this lab you will experiment with basic client-server programs.
    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You should be comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)

Note that this is the second step in this assignment. If you have not already created your slice, go to slice creation notebook or click [Here](./CreateSlice.ipynb)

## 1. Set up the Experiment


### 1.1  Retrieve Slice
Import the slice you created in the [Create Slice Notebook](./CreateSlice.ipynb).


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

In [ ]:
slice_name = "Lab01_Sockets"
slice = fablib.get_slice(slice_name)
slice.list_nodes()

In [ ]:
# Get geographic coordinates for each node
for node in slice.get_nodes():
    print(node.get_name())
    site = node.get_site()
    fablib.show_site(site, fields=['name','location'])

Make a note of the geographic coordinates of each of your nodes, as listed in the output tables above. **Record the coordinates in Lab1.docx**

### 1.2 Upload files
Upload test programs to each node.

In [ ]:
for node in slice.get_nodes():
    
    node.upload_file("testprogs/UDP_Echo_Client.py","UDP_Echo_Client.py")
    node.upload_file("testprogs/UDP_Echo_Server.py","UDP_Echo_Server.py")
    
    node.upload_file("testprogs/TCP_Echo_Client.py","TCP_Echo_Client.py")
    node.upload_file("testprogs/TCP_Echo_Server.py","TCP_Echo_Server.py")
    
    node.upload_file("testprogs/UDP_Ping_Client.py","UDP_Ping_Client.py")
    node.upload_file("testprogs/UDP_Ping_Server.py","UDP_Ping_Server.py")
    
    node.upload_file("testprogs/TCP_Ping_Client.py","TCP_Ping_Client.py")
    node.upload_file("testprogs/TCP_Ping_Server.py","TCP_Ping_Server.py")
    

## 2. Run Experiment

### 2.1 Getting Started

1. SSH into each node:
    - Go to the [Experiments](https://portal.fabric-testbed.net/experiments) page in the Fabric portal.
    - Click on the "telcom2310-fall23-babay" project
    - Click on "Slices" (on the lefthand side of the page)
    - Click on the "Lab01_Sockets" slice. This should bring up a visual representation of your topology.
    - For each node:
        - Click on that node and copy the provided "SSH Command". This should be something like `ssh -F <path to SSH config file> -i <path to private sliver key> ubuntu@205.172.170.122`
        - Open a terminal window (either on your local machine or in the JupyterHub) and enter the copied command, replacing `<path to SSH config file>` and `<path to private sliver key>` with the correct paths. For example, on JupyterHub, this would look like `ssh -F ~/work/fabric_config/ssh_config -i ~/work/fabric_config/fabric_sliver_key ubuntu@205.172.170.122`
        
        
2. Find the IP address for each node:
    - In the terminal of each node, enter the command `ip addr`.
    - You should see an output similar to the following (network addresses will differ):
    
        ```
        1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
            link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
            inet 127.0.0.1/8 scope host lo
               valid_lft forever preferred_lft forever
            inet6 ::1/128 scope host 
               valid_lft forever preferred_lft forever
        2: ens3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
            link/ether fa:16:3e:6b:8d:1c brd ff:ff:ff:ff:ff:ff
            inet 10.40.6.228/23 brd 10.40.7.255 scope global dynamic ens3
               valid_lft 73414sec preferred_lft 73414sec
            inet6 2620:0:c80:1003:f816:3eff:fe6b:8d1c/64 scope global dynamic mngtmpaddr noprefixroute 
               valid_lft 86397sec preferred_lft 14397sec
            inet6 fe80::f816:3eff:fe6b:8d1c/64 scope link 
               valid_lft forever preferred_lft forever
        3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
            link/ether 06:70:14:5e:a2:01 brd ff:ff:ff:ff:ff:ff
            inet 10.132.129.2/24 scope global ens7
               valid_lft forever preferred_lft forever
            inet6 fe80::470:14ff:fe5e:a201/64 scope link 
               valid_lft forever preferred_lft forever
        ```
    - Look at the entry for interface `ens7`. We will use its IPv4 address, which is the one following the word `inet`. In the example above, this would be `10.132.129.2`.
    - Note: if your entry for ens7 does not show an inet address, run the command `sudo ip link set dev ens7 up` and then re-run the `ip addr` command.
    - **Record the IP addresses of your client and server nodes in Lab1.docx**

### 2.2 Echo Programs

1. Review the code for the `UDP_Echo_Client.py` and `UDP_Echo_Server.py` programs (we will do this together in class). Note that you can use your preferred text editor on your own computer to view copies downloaded from Canvas. Or you can view them on a remote Fabric node using the command `cat <filename>` (e.g. `cat UDP_Echo_Server.py`) to print the file contents to your terminal window.

2. Run the UDP_Echo_Server.py program on your Server Node using the command:
```
python3 UDP_Echo_Server.py
```

3. Run the UDP_Echo_Client.py program on your Client Node using the command:
```
python3 UDP_Echo_Client.py -a <Node1_IP_Address>
```

where `<Node1_IP_Address>` is replaced with the IP address of your Server Node found in the “Getting Started” section above

You should be prompted to enter a message. Type in a word/sentence/phrase of your choice and hit enter.

The `UDP_Echo_Server.py` program running on your Server Node should receive the message, transform it to all UPPERCASE and echo it back to the client.
**Copy and paste the output from your server and client into Lab1.docx**

Note that the server will continue to wait for messages until it is manually killed. You can run the client as many times as you want (without stopping the server) and it should receive and echo the message back each time. To stop the server, use CTRL-C to kill the process.

4. Repeat steps 1-3 above for `TCP_Echo_Server.py` and `TCP_Echo_Client.py`
**Copy and paste the output from your server and client into Lab1.docx**

### 2.3 Ping Programs

1. Review the code for the UDP_Ping_Client.py and UDP_Ping_Server.py programs (we will do this together in class).

2. Run the UDP_Ping_Server.py program on your Server Node using the command:
```
python3 UDP_Ping_Server.py
```

3. Run the UDP_Ping_Client.py program on your Client Node using the command:
```
python3 UDP_Ping_Client.py -a <Node1_IP_Address>
```

where `<Node1_IP_Address>` is replaced with the IP address of your Server Node found in the “Getting Started” section above

By default, the client will perform 10 “pings”, where for each ping it: 1) sends a message to the server, 2) waits to receive a reply from the server, and 3) reports how long it took to send the message and receive the reply.

**Copy and paste the output from your server and client into Lab1.docx**

Note that you can change the number of pings performed by passing the `-n` commandline argument to the client program. For example, to do 20 pings:
```
python3 UDP_Ping_Client.py -a <Node1_IP_Address> -n 20
```

Note that the server will continue to wait for messages until it is manually killed. You can run the client as many times as you want (without stopping the server) and it should respond to the pings each time. To stop the server, use `CTRL-C` to kill the process.

4. Based on the geographic coordinates for your sites, calculate your expected UDP ping time. Use Google Maps to find the distance between your sites and assume the speed of light in fiber is 2x10^8 meters/second.

**Show your calculation in Lab1.docx. Compare your calculated value to the measured ping times. Do the results make sense? Explain.**

5. Repeat steps 1-3 above for TCP_Ping_Server.py and TCP_Ping_Client.py

**Copy and paste the output from your server and client into Lab1.docx**

6. Compare your results from running UDP and TCP Ping programs. What differences do you notice? Can you explain them based on what we’ve discussed in class?

**Answer in Lab1.docx**

### 2.4 Loss Emulation

Since we can’t predict whether we will actually encounter packet loss during our experiments, here we will artificially inject loss to examine its effects on our UDP and TCP ping programs

1. To create artificial loss, run the following command on your Server Node:
```
sudo tc qdisc add dev ens7 root netem loss 20%
```

Don’t worry about the details of this command – its effect is to randomly drop 20% of the packets leaving the server.

2. Run the UDP_Ping_Server.py program on your Server Node using the command:
```
python3 UDP_Ping_Server.py
```

3. Run the UDP_Ping_Client.py program, pinging 30 times on your GENI Node 2 using the command:
```
python3 UDP_Ping_Client.py -a <Node1_IP_Address> -n 30
```

where `<Node1_IP_Address>` is replaced with the IP address of your Server Node found in the “Getting Started” section above

**Copy and paste the output from your server and client into Lab1.docx**

To stop the server, use `CTRL-C` to kill the process

4. Repeat steps 2-3 above for TCP_Ping_Server.py and TCP_Ping_Client.py

**Copy and paste the output from your server and client into Lab1.docx**

5. Remove the emulated loss by running the following command on your Server Node:
```
sudo tc qdisc del dev eth0 root
```

6. Examine your results from running UDP and TCP Ping programs. What differences do you notice? Can you explain them based on what we’ve discussed in class?

**Answer in Lab1.docx**

### Bonus: Persistent TCP Connections

In our example TCP_Ping programs, notice that a new TCP connection is created for each ping that is performed. This is similar to how HTTP works when using **non-persistent connections**: a new connection is initiated for **each request+response pair**. In class, we discussed how **persistent connections** could be used instead so that **multiple requests/responses can be sent over the same TCP connection**.

To demonstrate the effect of this, you should create a TCP_Ping_Server_Persistent.py and TCP_Ping_Client_Persistent.py that modify the example TCP_Ping_Server.py and TCP_Ping_Client.py programs such that all ping+response messages for a single run of the client program are sent over the **same** TCP connection.

Hint 1: Start with the client side. This part should be fairly straightforward and involve moving the socket, connect, and close calls outside of the while loop.

Hint 2: After your client sends all of its ping messages (and gets responses for them) it will close the TCP connection. After this point, if the server calls “recv” on the connection socket again, it will return an empty message. In Python, you can check for this condition with:

```python
message = connection_socket.recv(RECV_BUFFER_SIZE)
if not message: # Check for empty message
    # Do something
```

Therefore, each time the server calls recv, it should check whether the connection was closed. If it was, then the server should also close the connection socket. At this point you can choose to quit the server (easier and perfectly fine for our purposes, but not quite as nice), or go back to listening for new connections to accept.

1. Run the TCP_Ping_Server_Persistent.py program on your Server using the command:
```
python3 TCP_Ping_Server_Persistent.py
```

2. Run the TCP_Ping_Client_Persistent.py program on your Client Node using the command:
```
python3 TCP_Ping_Client_Persistent.py -a <Node1_IP_Address>
```

where `<Node1_IP_Address>` is replaced with the IP address of your Server Node found in the “Getting Started” section above

**Copy and paste the output from your server and client into Lab1.docx**

3. Examine your results from running the persistent and non-persistent TCP Ping programs. What differences do you notice? Can you explain them based on what we’ve discussed in class?

**Answer in Lab1.docx**

## Cleanup Resources

Once you have completed the steps above, delete your slice to free up resources for other users. Note: if you stopped the notebook between running the first 3 code cells and getting to this point, you should re-run the first 2 code cells (but not the third) to retrieve the slice before running the following cell)

In [ ]:
try:
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")